In [ ]:
import os
import re
import jieba
import pandas as pd
from datetime import datetime, timezone
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
input_file = '/content/drive/My Drive/IRTM_final_project/data/posts/1220.csv'
output_file = '/content/drive/My Drive/IRTM_final_project/vectors/1220_vector.csv'

In [ ]:
from datetime import datetime

current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(current_time)

2024-12-20 08:53:11


In [ ]:
def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)

    text = re.sub(r'\d+', '', text)  # 移除數字

    # 使用正規表達式分詞
    tokens = re.findall(r'\b\w+\b', text.lower())  # 切割成單字，並轉為小寫
    english_tokens = [stemmer.stem(word) for word in tokens if word.isalpha() and word not in stop_words]

    # 中文處理
    chinese_tokens = jieba.lcut(text)

    # 合併英文和中文的 tokens
    combined_tokens = english_tokens + chinese_tokens
    return ' '.join(combined_tokens)

In [ ]:
def process_file(file, vectorizer, processed_texts):
    df = pd.read_csv(file)

    df = df[['id', 'text', 'reply_count', 'like_count']]

    # 使用緩存的 processed_text
    df['processed_text'] = processed_texts
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(current_time)

    tfidf_matrix = vectorizer.transform(df['processed_text'])
    vectors = tfidf_matrix.toarray()  # 轉換為 NumPy 陣列

    # 將每行的向量轉為列表格式，並儲存到 'vector' 欄位
    df['vector'] = [list(vector) for vector in vectors]

    # 保留 'id', 'reply_count', 'like_count' 及 'vector' 欄位
    result = df[['id', 'reply_count', 'like_count', 'vector']]

    # 儲存結果至檔案
    result.to_csv(output_file, index=False)
    print(f"前處理完成，結果儲存至 {output_file}")

In [ ]:
# 收集所有文本並處理一次
print("初始化 TF-IDF Vectorizer...")
df = pd.read_csv(input_file)
processed_texts = df['text'].apply(preprocess_text)

current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(current_time)

# 初始化 TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # 設定最大特徵數量
tfidf_vectorizer.fit(processed_texts)

current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(current_time)

# 使用處理後的文本進行向量化
print("開始處理 target.csv...")
process_file(input_file, vectorizer=tfidf_vectorizer, processed_texts=processed_texts)

初始化 TF-IDF Vectorizer...


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 0.689 seconds.
DEBUG:jieba:Loading model cost 0.689 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


2024-12-20 08:53:20
2024-12-20 08:53:22
開始處理 target.csv...
2024-12-20 08:53:22


In [ ]:
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(current_time)